Подготовить данные для модели:
    
   Анализ данных, необходимые корректировки. 
1. Обработать пропуски. 
2. Оценить выбросы. 
3. Корреляция. 
4. Тест на нормальность распределения.
5. Масштабировать данные.

Задача: есть данные, что тест на болезнь дает результат с вероятностью в 98%. Число больных в популяции у нас 1%. 
Вопрос - с какой вероятностью положительный тест будет положительным при случайном тестировании.  